In [4]:
# Task 1 Q1

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from scipy.stats import mode

# Defining the function to compute Jaccard similarity
def jaccard_similarity(a, b):
    union = np.sum(np.maximum(a, b))
    intersection = np.sum(np.minimum(a, b))
    return intersection / (union + 1) 

# Defining the K-means algorithm
def kmeans(X, k, similarity='options', max_iters=100):
    centroids = X[np.random.choice(len(X), k, replace=False)]

    for _ in range(max_iters):
        if similarity == 'euclidean':
            distances = pairwise_distances(X, centroids, metric='euclidean')
        elif similarity == 'cosine':
            distances = 1 - cosine_similarity(X, centroids)
        elif similarity == 'jaccard':
            distances = np.array([1 - jaccard_similarity(X[i], centroid) for i in range(len(X)) for centroid in centroids])
            distances = distances.reshape(len(X), k)

        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.sum(labels == i) > 0 else X[np.random.choice(len(X))] for i in range(k)])

        if np.all(np.abs(new_centroids - centroids) < 1e-6): 
            break

        centroids = new_centroids

    sse = np.sum((X - centroids[labels]) ** 2)

    return centroids, labels, sse

# Loading and standardizing the data
data = pd.read_csv('data.csv')
labelData = pd.read_csv('label.csv')
true_labels = labelData.values
scaler = StandardScaler()
features_standardized = scaler.fit_transform(data.values)
features = data.values

# Implementing K-means using different similarity metrics
k = len(labelData['7'].unique())
centroids_euclidean, labels_euclidean, sse_euclidean = kmeans(features, k, similarity='euclidean')
centroids_cosine, labels_cosine, sse_cosine = kmeans(features, k, similarity='cosine')
centroids_jaccard, labels_jaccard, sse_jaccard = kmeans(features, k, similarity='jaccard')

# Printing the results
print(f"SSE (Euclidean-K-means): {sse_euclidean}")
print(f"SSE (Cosine-K-means): {sse_cosine}")
print(f"SSE (Jaccard-K-means): {sse_jaccard}")


SSE (Euclidean): 25429779434.864655
SSE (Cosine): 25651039252.47297
SSE (Jaccard): 25596447671.65693


In [5]:
# Task 1 Q2

from sklearn.metrics import accuracy_score

# Defining a function to label clusters based on majority vote
def label_clusters(labels, true_labels):
    cluster_labels = []
    for cluster_id in np.unique(labels):
        cluster_indices = np.where(labels == cluster_id)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_label = mode(cluster_true_labels)[0][0]
        cluster_labels.append((cluster_id, majority_label))
    return cluster_labels

# Defining a function to assign predicted labels based on majority vote for each cluster
def assign_predicted_labels(labels, cluster_labels):
    predicted_labels = np.zeros_like(labels)
    for cluster_id, majority_label in cluster_labels:
        predicted_labels[labels == cluster_id] = majority_label
    return predicted_labels

# Computing accuracy for each clustering method
def compute_accuracy(true_labels, predicted_labels):
    return accuracy_score(true_labels, predicted_labels)

# Labeling clusters for each clustering method
cluster_labels_euclidean = label_clusters(labels_euclidean, true_labels)
cluster_labels_cosine = label_clusters(labels_cosine, true_labels)
cluster_labels_jaccard = label_clusters(labels_jaccard, true_labels)

# Assigning predicted labels based on majority vote for each clustering method
predicted_labels_euclidean = assign_predicted_labels(labels_euclidean, cluster_labels_euclidean)
predicted_labels_cosine = assign_predicted_labels(labels_cosine, cluster_labels_cosine)
predicted_labels_jaccard = assign_predicted_labels(labels_jaccard, cluster_labels_jaccard)

# Computing accuracy for each clustering method
accuracy_euclidean = compute_accuracy(true_labels, predicted_labels_euclidean)
accuracy_cosine = compute_accuracy(true_labels, predicted_labels_cosine)
accuracy_jaccard = compute_accuracy(true_labels, predicted_labels_jaccard)

# Printing the results
print(f"Accuracy (Euclidean-K-means): {accuracy_euclidean}")
print(f"Accuracy (Cosine-K-means): {accuracy_cosine}")
print(f"Accuracy (Jaccard-K-means): {accuracy_jaccard}")


Accuracy (Euclidean-K-means): 0.6048604860486049
Accuracy (Cosine-K-means): 0.5797579757975797
Accuracy (Jaccard-K-means): 0.5512551255125513


In [6]:
# Task 1 Q3

import time

# Defining modified K-means algorithm with stop criteria
def kmeans_with_stop_criteria(X, k, similarity='euclidean', max_iters=100, max_iter_without_change=500):
    centroids = X[np.random.choice(len(X), k, replace=False)]
    prev_sse = np.inf
    iter_without_change = 0

    start_time = time.time()

    for iter_count in range(max_iters):
        if similarity == 'euclidean':
            distances = pairwise_distances(X, centroids, metric='euclidean')
        elif similarity == 'cosine':
            distances = 1 - cosine_similarity(X, centroids)
        elif similarity == 'jaccard':
            distances = np.array([1 - jaccard_similarity(X[i], centroid) for i in range(len(X)) for centroid in centroids])
            distances = distances.reshape(len(X), k)

        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.sum(labels == i) > 0 else X[np.random.choice(len(X))] for i in range(k)])

        sse = np.sum((X - centroids[labels]) ** 2)

        if sse >= prev_sse:
            iter_without_change += 1
        else:
            iter_without_change = 0

        if np.all(np.abs(new_centroids - centroids) < 1e-6) or sse >= prev_sse or iter_without_change >= max_iter_without_change:
            break

        centroids = new_centroids
        prev_sse = sse

    end_time = time.time()
    execution_time = end_time - start_time

    return centroids, labels, sse, iter_count + 1, execution_time

# Applying modified K-means with stop criteria for each similarity metric
k = len(np.unique(true_labels))
centroids_euclidean, labels_euclidean, sse_euclidean, iter_count_euclidean, exec_time_euclidean = kmeans_with_stop_criteria(features, k, similarity='euclidean')
centroids_cosine, labels_cosine, sse_cosine, iter_count_cosine, exec_time_cosine = kmeans_with_stop_criteria(features, k, similarity='cosine')
centroids_jaccard, labels_jaccard, sse_jaccard, iter_count_jaccard, exec_time_jaccard = kmeans_with_stop_criteria(features, k, similarity='jaccard')

# Printing the results
print("Euclidean-K-means:")
print(f"Number of iterations: {iter_count_euclidean}")
print(f"Execution time: {exec_time_euclidean} seconds\n")

print("Cosine-K-means:")
print(f"Number of iterations: {iter_count_cosine}")
print(f"Execution time: {exec_time_cosine} seconds\n")

print("Jaccard-K-means:")
print(f"Number of iterations: {iter_count_jaccard}")
print(f"Execution time: {exec_time_jaccard} seconds\n")


Euclidean-K-means:
Number of iterations: 67
Execution time: 24.824869632720947 seconds

Cosine-K-means:
Number of iterations: 35
Execution time: 16.00656032562256 seconds

Jaccard-K-means:
Number of iterations: 26
Execution time: 78.13513684272766 seconds



In [8]:
# Task 1 Q4

# Defining modified K-means function with specified stopping criteria
def kmeans_with_termination(X, k, similarity='euclidean', max_iters=100, max_iterations=500):
    centroids = X[np.random.choice(len(X), k, replace=False)]
    prev_sse = np.inf

    for iteration in range(max_iterations):
        if similarity == 'euclidean':
            distances = pairwise_distances(X, centroids, metric='euclidean')
        elif similarity == 'cosine':
            distances = 1 - cosine_similarity(X, centroids)
        elif similarity == 'jaccard':
            distances = np.array([1 - jaccard_similarity(X[i], centroid) for i in range(len(X)) for centroid in centroids])
            distances = distances.reshape(len(X), k)

        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.sum(labels == i) > 0 else X[np.random.choice(len(X))] for i in range(k)])
        sse = np.sum((X - new_centroids[labels]) ** 2)

        if np.all(np.abs(new_centroids - centroids) < 1e-6) or sse > prev_sse or iteration == max_iters:
            break

        centroids = new_centroids
        prev_sse = sse

    return centroids, labels, sse, iteration + 1

# Defining a function to compare SSEs under different terminating conditions
def compare_sses(X, k, similarity='euclidean', max_iters=100, max_iterations=500):
    sses = []

    for condition in ['no_change', 'sse_increase', 'max_iterations']:
        _, _, sse, iterations = kmeans_with_termination(X, k, similarity, max_iters, max_iterations)
        sses.append((condition, sse, iterations))

    return sses

# Computing SSEs for Euclidean-K-means, Cosine-K-means, and Jaccard-K-means under different terminating conditions
sse_euclidean = compare_sses(features, k, similarity='euclidean')
sse_cosine = compare_sses(features, k, similarity='cosine')
sse_jaccard = compare_sses(features, k, similarity='jaccard')

# Printing the results
print("Euclidean-K-means SSEs:")
for condition, sse, iterations in sse_euclidean:
    print(f"Condition: {condition}, SSE: {sse}, Iterations: {iterations}")

print("\nCosine-K-means SSEs:")
for condition, sse, iterations in sse_cosine:
    print(f"Condition: {condition}, SSE: {sse}, Iterations: {iterations}")

print("\nJaccard-K-means SSEs:")
for condition, sse, iterations in sse_jaccard:
    print(f"Condition: {condition}, SSE: {sse}, Iterations: {iterations}")
    

Euclidean-K-means SSEs:
Condition: no_change, SSE: 25322242913.74782, Iterations: 80
Condition: sse_increase, SSE: 25517947871.9446, Iterations: 46
Condition: max_iterations, SSE: 25578429886.357285, Iterations: 40

Cosine-K-means SSEs:
Condition: no_change, SSE: 25623047296.12691, Iterations: 37
Condition: sse_increase, SSE: 25622744459.17241, Iterations: 24
Condition: max_iterations, SSE: 25424768296.929214, Iterations: 39

Jaccard-K-means SSEs:
Condition: no_change, SSE: 25424517815.049168, Iterations: 14
Condition: sse_increase, SSE: 25412038652.99215, Iterations: 37
Condition: max_iterations, SSE: 25507576353.208992, Iterations: 32
